# These are the libraries we have used for Face Recognition

In [1]:
import face_recognition
import cv2
import os
import numpy as np 

# Now we are declaring 2 empty lists for storing our face encodings and names respectively 

In [2]:
# Load the known faces and their names
known_face_encodings = []
known_face_names = [] 

# This is the variable which stores the path to our database, which is our image folder

In [3]:
# Load images and encode them
image_path = r"C:\Users\HP\OneDrive\Desktop\Face Detection\Database"
image_files = [f for f in os.listdir(image_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
for image_file in image_files:
    image = face_recognition.load_image_file(os.path.join(image_path, image_file))
    encoding = face_recognition.face_encodings(image)[0]  # Assuming there's only one face per image
    known_face_encodings.append(encoding)
    name = os.path.splitext(image_file)[0]  # Use the filename as the person's name
    known_face_names.append(name)

# Initialize the Webcam
video_capture = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not video_capture.isOpened():
    print("Failed to open webcam. Make sure it's connected and not being used by another application.")
    exit()

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_frame = frame[:, :, ::-1]

    # Find all the face locations in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)

    # Loop through each face found in the frame
    for face_location in face_locations:
        top, right, bottom, left = face_location
        # Resize the face image to a standard size (150x150 pixels)
        face_image = cv2.resize(frame[top:bottom, left:right], (150, 150))
        face_encoding = face_recognition.face_encodings(face_image)

        # See if the face is a match for the known face(s)
        if len(face_encoding) > 0:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding[0])
            name = "Unknown"

            # If a match is found, use the name of that person
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()